# Práctica de SQL

## Generación de datos

Ejecutamos el simulador con el siguiente comando:
    
```
./bin/eventsim -c examples/example-config.json  -n 1000 -t 3 events
```

Eso nos va a generar varios archivos en formato JSONLine

In [17]:
import pandas as pd
import psycopg2 as pg
connection = pg.connect("host=localhost user=catedra password=S3cret")
def sql(text):
    return pd.read_sql(text, con=connection)

In [18]:
sql("SELECT * FROM page_view_events")

,ts,sessionid,level,iteminsession,city,zip,state,useragent,lon,lat,...,gender,registration,page,auth,method,status,trackid,artist,song,duration
0,1704548537817,1693,paid,2,Tucson,85710,AZ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-110.824,32.214,...,M,1.704548e+12,NextSong,Logged In,PUT,200,TRJHAGN128E078506F,Skillet,A Little More (Album Version),289.67140
1,1704548545817,213,paid,35,Davis,95616,CA,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10)...",-121.798,38.554,...,F,1.704548e+12,NextSong,Logged In,PUT,200,TRFDJKM128F92EE287,Philippe Rochard,Crumpshit,360.51547
2,1704548618817,893,paid,77,Hanover Park,60133,IL,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",-88.143,41.978,...,F,1.704548e+12,NextSong,Logged In,PUT,200,TRONYHY128F92C9D11,Kings Of Leon,Revelry,201.79546
3,1704548630817,598,free,0,Laredo,78045,TX,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-99.685,27.834,...,F,1.704548e+12,Home,Logged In,GET,200,None,None,None,NaN
4,1704548649817,598,free,1,Laredo,78045,TX,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-99.685,27.834,...,F,1.704548e+12,NextSong,Logged In,PUT,200,TRQNZBB12903D0EA7B,Ilario Alicante,Vacaciones en Chile,643.29095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25978,1705585224817,5670,free,1,Norwood,2062,MA,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-71.196,42.187,...,M,1.704548e+12,NextSong,Logged In,PUT,200,TRXSXDQ128F426643E,REO Speedwagon,(I Believe) Our Time Is Gonna Come,302.99384
25979,1705585231817,2924,paid,1,Mchenry,60051,IL,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:30.0) G...,-88.229,42.355,...,F,1.704548e+12,NextSong,Logged In,PUT,200,TRJQEQC128F93607C7,Juaninacka,EspaÃÂÃÂ±a_ Sevilla y yo,202.86649
25980,1705585238817,4279,paid,4,Estill Springs,37330,TN,"""Mozilla/5.0 (iPhone; CPU iPhone OS 7_1_2 like...",-86.131,35.280,...,M,1.704548e+12,NextSong,Logged In,PUT,200,TRVPSZF128F147DF97,Coldplay,We Never Change,249.10323
25981,1705585280817,5551,paid,8,Orlando,32803,FL,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",-81.346,28.556,...,F,1.704548e+12,NextSong,Logged In,PUT,200,TRAVMND128F4275780,Katrina & The Waves,Walking On Sunshine,220.91710


## Consultas

### Breve repaso de la DML de Postgres

A continuación detallamos de de manera simplificada la gramática de las consultas del Data Manipulation Language de Postgres. Para más detalle ver la documentación [oficial](https://www.postgresql.org/docs/current/queries.html)). Es importante siempre destacar que cada base de datos puede implementar un subconjunto o superconjunto de SQL, por lo que la gramática que contamos a continuación puede ser un poco distinta en otros motores.

Como es estándar en las documentaciones, marcamos que las expresiones entre corchetes `[]` son opcionales, y cuando usamos elipsis `[, ...]` admitimos aplicación recursiva de la expresión. Por ejemplo, el `[WITH table_name AS (select_expression) [, ...]]` se puede expandir a `WITH table_name1 AS (select_expression1), table_name2 AS (select_expression2), table_name3 AS (select_expression3)` y así sucesivamente.

La gramática relevante para el propósito de nuestra práctica es la siguiente:

```sql
[WITH table_name AS (select_expression) [, ...] ] 
SELECT [DISTINCT [ON (expression, [, expression ...])]]
    select_list 
FROM 
    table_reference [, ...]
[WHERE condition]
[
 GROUP BY grouping_column_reference [, ...]
 [HAVING boolean_expression]
]
[ORDER BY ...]
[LIMIT number]
[OFFSET number ]
```

En la primera práctica vamos a enfocarnos en consultas `SELECT` con distintos filtros y joins, mientras que en la segunda clase avanzaremos sobre mecanismos de agrupación de datos y consultas sobre grupos.

¡Siempre es bueno agregar comentarios en el código! Para eso podemos usar el indicador `--`.

### Consultas con proyecciones y filtros

Muchas veces para entender los datos sirve traerse una muestra de todas las columnas y algunas filas:

In [19]:
sql("""
SELECT 
    * 
FROM listen_events
LIMIT 5
""")

,sessionid,iteminsession,userid,ts,auth,level,trackid,song,artist,zip,city,state,useragent,lon,lat,lastname,firstname,gender,registration,duration
0,1693,2,472,1704548537817,Logged In,paid,TRJHAGN128E078506F,A Little More (Album Version),Skillet,85710,Tucson,AZ,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-110.824,32.214,Reyes,Tyrique,M,1704548482817,289.67140
1,213,35,214,1704548545817,Logged In,paid,TRFDJKM128F92EE287,Crumpshit,Philippe Rochard,95616,Davis,CA,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10)...",-121.798,38.554,Krause,Olivia,F,1704548482817,360.51547
2,893,77,894,1704548618817,Logged In,paid,TRONYHY128F92C9D11,Revelry,Kings Of Leon,60133,Hanover Park,IL,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",-88.143,41.978,Arnold,Kierra,F,1704548482817,201.79546
3,598,1,599,1704548649817,Logged In,free,TRQNZBB12903D0EA7B,Vacaciones en Chile,Ilario Alicante,78045,Laredo,TX,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",-99.685,27.834,Trujillo,Shantel,F,1704548482817,643.29095
4,552,43,553,1704548654817,Logged In,paid,TRLBBFR128F9334018,Never Gonna Happen,Lily Allen,78046,Laredo,TX,Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; r...,-99.351,27.374,Wheeler,Josephine,F,1704548482817,207.17670


O particularmente algunas columnas:

In [20]:
sql("""
SELECT 
    artist, song, level
FROM listen_events
LIMIT 5
""")

,artist,song,level
0,Skillet,A Little More (Album Version),paid
1,Philippe Rochard,Crumpshit,paid
2,Kings Of Leon,Revelry,paid
3,Ilario Alicante,Vacaciones en Chile,free
4,Lily Allen,Never Gonna Happen,paid


A veces es necesario renombrar las columnas para algún uso en particular:

In [21]:
sql("""
SELECT 
    artist, song, level AS tier
FROM listen_events
LIMIT 5
""")

,artist,song,tier
0,Skillet,A Little More (Album Version),paid
1,Philippe Rochard,Crumpshit,paid
2,Kings Of Leon,Revelry,paid
3,Ilario Alicante,Vacaciones en Chile,free
4,Lily Allen,Never Gonna Happen,paid


Podemos filtrar los datos:

In [22]:
sql("""
SELECT 
    artist, song, level AS tier
FROM listen_events
WHERE artist = 'Lily Allen'
LIMIT 5
""")

,artist,song,tier
0,Lily Allen,Never Gonna Happen,paid
1,Lily Allen,Friday Night,free
2,Lily Allen,LDN (Switch Remix),free
3,Lily Allen,Smile (Explicit Version),paid
4,Lily Allen,Not Big,paid


También podemos usar varias condiciones:

In [23]:
sql("""
SELECT 
    artist, song, level AS tier
FROM listen_events
-- Notar que no podemos usar el alias en where o en having
WHERE (artist = 'Lily Allen' AND level = 'free')
LIMIT 5
""")

,artist,song,tier
0,Lily Allen,Friday Night,free
1,Lily Allen,LDN (Switch Remix),free
2,Lily Allen,Not Big,free
3,Lily Allen,Cheryl Tweedy,free
4,Lily Allen,Cheryl Tweedy,free


### Usando funciones de agregación

Duración promedio de las canciones escuchadas:

In [24]:
sql("""
SELECT 
    AVG(duration) -- Podemos incluir "AS avg_duration" para ser más prolijos
FROM listen_events
""")

,avg
0,248.443902


Sesión máxima:

In [26]:
sql("""
SELECT 
    MAX(itemInSession) AS max_session_length
FROM page_view_events
""")

,max_session_length
0,185


### Agrupando los datos

In [27]:
sql("""
SELECT 
    page, COUNT(*) AS num_page_views
FROM page_view_events
GROUP BY page
""")

,page,num_page_views
0,Save Settings,37
1,Settings,158
2,Logout,283
3,Help,182
4,Cancellation Confirmation,1
5,Error,22
6,Submit Downgrade,1
7,Home,2921
8,About,121
9,NextSong,21596


### Filtrando grupos en base a propiedades agregadas

Queremos saber cuáles son las sesiones que tienen más de 20 canciones escuchadas, y para ellas, los usuarios:

In [ ]:
sql("""
SELECT 
    sessionId, 
    lastName, 
    firstName,
    MAX(itemInsession)
FROM listen_events
GROUP BY sessionId, lastName, firstName
HAVING MAX(itemInSession) > 20
""")

,sessionid,lastname,firstname,max
0,3375,Ramirez,Kira,59
1,3505,Reyes,Tyrique,87
2,2684,Colon,Rhys,42
3,690,Davis,Aniyah,33
4,2448,Cruz,William,55
...,...,...,...,...
275,2388,Mullins,Monica,41
276,4417,Brown,Paris,24
277,2071,Barron,Tyler,52
278,3188,Thompson,Maddox,55


### Implementación de paginación

Supongamos que queremos buscar el historial para un usuario de reproducciones. Para eso podemos usar un simple filtro WHERE, pero:

* eso nos traería todos los registros potencialmente en cualquier orden. Normalmente queremos mostrar lo más reciente primero. Para eso usamos un `ORDER BY COL DESC`. 
* por otra parte, no queremos traer todos los registros, sólo los que son necesarios para mostrar. Para eso podemos usar un `LIMIT 10`.
* pero, ¿cómo hacemos para obtener la siguiente página tras haber usado un `LIMIT`? Una solucion
    * podemos usar la función `OFFSET N`, que ignora las primeras `N` filas. Entonces `LIMIT N OFFSET K*N` nos trae la página K-ésima.

In [ ]:
sql("""
SELECT
    artist,
    song,
    ts,
    userId
FROM listen_events
WHERE
    userId = 421
ORDER BY ts DESC
LIMIT 10
OFFSET 10
""")

,artist,song,ts,userid
0,Boys Noize,Oh!,1705569355817,421
1,Boys Noize,Oh!,1705569355817,421
2,Kings Of Leon,Revelry,1705569154817,421
3,Kings Of Leon,Revelry,1705569154817,421
4,Kings Of Leon,Revelry,1705569154817,421
5,The Killers,Bling (Confession Of A King),1705568906817,421
6,The Killers,Bling (Confession Of A King),1705568906817,421
7,The Killers,Bling (Confession Of A King),1705568906817,421
8,Static-X,Destroy All (Album Version),1705568768817,421
9,Static-X,Destroy All (Album Version),1705568768817,421


Pendientes
* describir más las consultas
* una con select distinct?
* 
%md 
Realizar consultas básicas con las cláusulas SELECT, FROM, WHERE, GROUP BY, HAVING. Ordenamiento y paginación.

* Proyección: seleccionar sólo artista y canción
* Agrupamiento: contar las canciones más escuchadas
    * 

## Ejercicios extra

Pendiente: enumerar algunas consultas a realizar.

## Mejores prácticas para escribir SQL

### Antipatrones de consultas

### Guía de formato para consultas SQL